## Grading Rubric
### Business Understanding (10 points total).

• Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). Describe how you would define and measure the outcomes from the dataset. That is, why is this data important and how do you know if you have mined useful knowledge from the dataset? How would you measure the effectiveness of a good prediction algorithm? Be specific.

(Nnenna)


We are predicting Injury Severity 

### Data Understanding (80 points total)

#### Data Meaning Type
• [10 points] Describe the meaning and type of data (scale, values, etc.) for each
attribute in the data file.

(Nnenna)
#### Data Quality
• [15 points] Verify data quality: Explain any missing values, duplicate data, and outliers.
Are those mistakes? How do you deal with these problems? Be specific.

(Satvik and Dylan)
Data cleaning - added features, cleaned

scatter plots

#### Simple Statistics
• [10 points] Give simple, appropriate statistics (range, mode, mean, median, variance,
counts, etc.) for the most important attributes and describe what they mean or if you found something interesting. Note: You can also use data from other sources for comparison. Explain the significance of the statistics run and why they are meaningful.

(Dylan)
describe()

#### Visualize Attributes
• [15 points] Visualize the most important attributes appropriately (at least 5 attributes). Important: Provide an interpretation for each chart. Explain for each attribute why the chosen visualization is appropriate.

bar graph top makes or model

proportion plots - classification

Satvik

#### Explore Joint Attributes
• [15 points] Visualize relationships between attributes: Look at the attributes via scatter plots, correlation, cross-tabulation, group-wise averages, etc. as appropriate. Explain any interesting relationships.

(Jobin)
boxplots - injury severity

correlation

violin plot 




#### Explore Attributes and Class
• [10 points] Identify and explain interesting relationships between features and the class you are trying to predict (i.e., relationships with variables and the target classification).

Accuracy, precision, recall - categorical
ROC Curves, AUC

RMSE
(Jobin)

#### New Features
• [5 points] Are there other features that could be added to the data or created from existing features? Which ones?

Copy from earlier, use some columns for another dataset.

Dylan/Satvik

#### Exceptional Work (10 points total)
• You have free reign to provide additional analyses.
• One idea: implement dimensionality reduction, then visualize and interpret the results.  

# Business Understanding

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')

import plotly.express as px
import plotly.graph_objects as go

In [2]:
aviation_data = pd.read_csv("Data/AviationData.csv")
aviation_data.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'LOCATION', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

Need to convert Total Injuries to integer, clean up fails, remove 8 engine NASA flight, drop the remaining columns, Injury Severity, that we are going to remove:

Event.db, Accident.Number, Far.Description, Longitude, Latitude, Airport Code, Total Serious Injuries, Total.Minor.Injuries, Total.Fatal.Injuries, Publication.Date, Registration.Number, LOCATION, Accident.Number


Total Injuries = Total Serious Injuries + Total.Minor.Injuries + Total.Fatal.Injuries
Location = City, State



Report.Status?

In [3]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85976 entries, 0 to 85975
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                85976 non-null  object 
 1   Investigation.Type      85963 non-null  object 
 2   Accident.Number         85976 non-null  object 
 3   Event.Date              85976 non-null  object 
 4   LOCATION                85898 non-null  object 
 5   Country                 85469 non-null  object 
 6   Latitude                31587 non-null  float64
 7   Longitude               31578 non-null  float64
 8   Airport.Code            48612 non-null  object 
 9   Airport.Name            51298 non-null  object 
 10  Injury.Severity         85842 non-null  object 
 11  Aircraft.damage         83047 non-null  object 
 12  Aircraft.Category       29226 non-null  object 
 13  Registration.Number     81756 non-null  object 
 14  Make                    85908 non-null

In [4]:
aviation_data.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,LOCATION,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,2.02102E+13,Accident,CEN21FA130,2021-02-16,"JANESVILLE, WI",United States,42.595377,-89.030245,NaN,NaN,...,Ferry,NaN,2.0,0.0,0.0,0.0,VMC,NaN,NaN,NaN
1,2.02102E+13,Accident,ERA21FA130,2021-02-15,"St Thomas, CB",United States,18.354444,-65.027778,NaN,NaN,...,Aobv,Caribbean Buzz Management Llc.,4.0,0.0,0.0,0.0,VMC,NaN,NaN,NaN
2,2.02102E+13,Accident,ANC21LA017,2021-02-13,"TYONEK, AK",United States,61.336392,-152.016430,NaN,NaN,...,Personal,Paul Andrews,0.0,0.0,2.0,0.0,NaN,NaN,NaN,NaN
3,2.02102E+13,Accident,CEN21LA127,2021-02-12,"PRAIRIE DU SAC, WI",United States,43.297731,-89.755693,91C,SAUK-PRAIRIE,...,Instructional,NaN,0.0,0.0,0.0,1.0,VMC,NaN,NaN,NaN
4,2.02102E+13,Accident,ERA21LA131,2021-02-10,"LAKE PLACID, FL",United States,27.243723,-81.413767,09FA,NaN,...,Personal,Case Robert,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN


In [5]:
#splitting state and city from location
aviation_data['City'] = aviation_data['LOCATION'].str.split(',').str[0]
aviation_data['State'] = aviation_data['LOCATION'].str.split(',').str[1]
#dropping location since we now have state and city

In this section, we will verify the data quality, explain missing values, duplicate data, and outliers. We will also deal with missing values and address any points of interest.

Something we would like to do before looking at missing values is make the data easier to read by adding some columns. We will split the "LOCATION" column into "City" and "State"

Next, we will add a column called "Total Injuries" which is the sum of Total Fatal Injuries, Total Minor Injuries and Total Serious Injuries.


In [6]:
#adding a total injuries column
aviation_data["Total_Injuries"] = aviation_data['Total.Fatal.Injuries'] \
                            + aviation_data['Total.Serious.Injuries'] \
                            + aviation_data['Total.Minor.Injuries']


# Data Meaning/Type

In [7]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85976 entries, 0 to 85975
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                85976 non-null  object 
 1   Investigation.Type      85963 non-null  object 
 2   Accident.Number         85976 non-null  object 
 3   Event.Date              85976 non-null  object 
 4   LOCATION                85898 non-null  object 
 5   Country                 85469 non-null  object 
 6   Latitude                31587 non-null  float64
 7   Longitude               31578 non-null  float64
 8   Airport.Code            48612 non-null  object 
 9   Airport.Name            51298 non-null  object 
 10  Injury.Severity         85842 non-null  object 
 11  Aircraft.damage         83047 non-null  object 
 12  Aircraft.Category       29226 non-null  object 
 13  Registration.Number     81756 non-null  object 
 14  Make                    85908 non-null

# Verify Data Quality

In [8]:
aviation_data.head()

NameError: name 'aviation_merged' is not defined

In [ ]:
#We have 30 columns to work with
#First we will check to see what percent of each column is null

#Percent of missing data
percent_missing = aviation_data.isnull().sum() * 100 / len(aviation_data)
#Create DF 
missing_value_df = pd.DataFrame({'column_name': aviation_data.columns,
                                 'percent_missing': percent_missing})
#Then sort by least to most
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

What we can see from the missing data above is categories such as Air Carrier and schedule are missing the most. When prodicting with this dataset we will primarly forcus on total number of injuries and Injury severity. Something to consider with the missing data in air carrier is if there was a corrlation between air carrier and plane crashes I don't beleive that business would still be operating.

Regarding many of the missing values in the fields: Total Fatal Injuries, Total Minor Injuries and Total Serious Injuries: we will be adding a total injuries column and consult outside sources to confirm these nulls as 0s. 

Need to convert Total Injuries to integer, clean up fails, remove 8 engine NASA flight, drop the remaining columns, Injury Severity, that we are going to remove:

Event.Ib, Accident.Number, Far.Description, Longitude, Latitude, Airport Code, Total Serious Injuries, Total Serious Injuries, Total.Fatal.Injuries, Publication.Date, Registration.Number, LOCATION, Accident.Number, Schedule


Total Injuries = Total Serious Injuries + Total.Minor.Injuries + Total.Fatal.Injuries
Location = City, State

rc_aviation = rc_aviation.drop(columns=['FAR.Description', 'Event.Id',"Accident.Number","Longitude",
                                        "Latitude","Airport.Code","Total.Serious.Injuries","Total.Minor.Injuries",
                                        "Total.Fatal.Injuries","Publication.Date","Registration.Number","LOCATION",
                                        "Accident.Number","Schedule", "LOCATION", "Event.Date"])

In [ ]:
#dropping FAR.Descripiton, schedule, air.carrier due to a lack of data
#dropping location as it is redundant to city, state and country
#also dropping column named "text" I am not sure how that got into our df
rc_aviation = aviation_data.copy()
rc_aviation = rc_aviation.drop(columns=['FAR.Description', 'Event.Id',"Accident.Number","Longitude",
                                        "Latitude","Airport.Code","Publication.Date","Registration.Number","LOCATION",
                                        "Accident.Number","Schedule", "LOCATION", "Event.Date"])
rc_aviation.columns

In [ ]:
#checking for duplicates
#before we step into ourliars we would like to look for duplicates
#using the field acident number we will search for duplicates

rc_aviation[rc_aviation.duplicated(subset=['Accident.Number'])]


In [ ]:
#now that we see we have a duplicate lets compare them
rc_aviation[rc_aviation['Accident.Number']=='ERA20CA175']

In [ ]:
#Summary of the data for continious variables
#here we are looking for outliars and to see if median or mean can be used to fill in missing values
rc_aviation.describe().apply(lambda s: s.apply('{0:.1f}'.format))

# Simple Statisitics

In [ ]:
#looking at accidents by Country
aviation_country = rc_aviation['Country'].groupby(rc_aviation['Country']).count()
aviation_country


In [ ]:
# Lets plot on a map where total injuries happened
aviation_merged['text'] = aviation_merged['Country'] + '<br>Total Injuries ' + (aviation_merged['Total Injuries'].astype(str))
colors = ["darkblue","yellow","seagreen","purple","cyan","orange"]
limits = [(0,50),(50,100),(100,150),(150,175),(175,200),(350,400)]
#scale = 10
accidentState = []

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    Injuries_Count = aviation_merged[((aviation_merged['Total Injuries'] > lim[0]) & (aviation_merged['Total Injuries'] <lim[1]))]
    fig.add_trace(go.Scattergeo(
        locationmode = 'country names',
        lon = Injuries_Count['Longitude'],
        lat = Injuries_Count['Latitude'],
        opacity = 0.8,
        text = Injuries_Count['text'],
        marker = dict(
            size = Injuries_Count['Total Injuries'],
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))
    accidentState.append(Injuries_Count)

fig.update_layout(
        title_text = 'Distribution of Total Injuries in the World<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope = 'world',
            landcolor = 'rgb(217, 217, 217)',
        )
    )
fig.show()